In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import html5lib
import sqlite3

In [2]:
# headless

In [3]:
def save_csv(df,name):

    name += ".csv"
    df.to_csv(name,index=False)
    
    return "succes"

def save_excel(df,name):

    name += ".xlsx"
    df.to_excel(name,index=False)
    
    return "succes"

In [11]:
class SET_Schap:
    
    def __init__(self):
        
        self.location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V2.sqlite'
        self.Industrys = ['AGRO','CONSUMP','FINCIAL','INDUS','PROPCON','RESOURC','SERVICE','TECH']
        self.Sector = ['AGRI', 'FOOD', 'FASHION', 'HOME', 'PERSON', 'BANK', 'FIN', 'INSUR', 'AUTO', 'IMM', 'PAPER', 'PETRO', 'PKG', 'STEEL', 'CONMAT', 'PROP', 'PF&REIT', 'CONS', 'ENERG', 'MINE', 'COMM', 'HELTH', 'MEDIA', 'PROF', 'TOURISM', 'TRANS', 'ETRON', 'ICT']
        self.driver = webdriver.Chrome(r"C:/Users/JourneyQ/OneDrive - kmutnb.ac.th/Desktop/Quick_file/year_2-S_2/Project_sofedev_2/Schap/chromedriver.exe")
        self.driver.get("https://www.set.or.th/th/market/index/set/overview")
        self.symbol_id =  self.get_symbol_id()
        
    def get_symbol_id(self):
        
        con = sqlite3.connect(self.location_db)
        cur = con.cursor()
        
        sql = """select Symbol,SymbolId 
                from Information
                """
        cur.execute(sql)
        records = cur.fetchall()
        con.close()
        symbol_id = {}
        
        for share in records:
            
            symbol_id[share[0]] = share[1]
        
        return symbol_id
    
    def get_set50_100_df(self):

        self.driver.get("https://www.set.or.th/en/market/index/set100/overview")
        data = self.driver.page_source
        data_df = pd.read_html(data)[2]
        clean_column = []
        for column in data_df.columns:

            clean_column.append(column.replace("  (Click to sort Ascending)",""))

        data_df.columns = clean_column
        data_df = data_df.set_index('Symbol')
        return data_df 

    def get_share(self,data_df,all_share):

        Section_Industry = ['AGRO', 'AGRI', 'FOOD', 'CONSUMP', 'FASHION', 'HOME', 'PERSON', 'FINCIAL', 'BANK', 'FIN', 'INSUR', 'INDUS', 'AUTO', 'IMM', 'PAPER', 'PETRO', 'PKG', 'STEEL', 'PROPCON', 'CONMAT', 'PROP', 'PF&REIT', 'CONS', 'RESOURC', 'ENERG', 'MINE', 'SERVICE', 'COMM', 'HELTH', 'MEDIA', 'PROF', 'TOURISM', 'TRANS', 'TECH', 'ETRON', 'ICT']

        for i in data_df['หลักทรัพย์']:

            if str(i) != 'nan' and not(str(i.split()[0]) in Section_Industry):
#                 print(i.split()[0])
                all_share.append(i.split()[0])

        return all_share

    def get_SET_share(self):

        Industrys = self.Industrys
        all_share = []

        for Industry in Industrys:

            self.driver.get("https://www.set.or.th/th/market/index/set/"+Industry)
            self.driver.execute_script("scrollBy(0,+5000);")
            time.sleep(2)

            data = self.driver.page_source
            data_df = pd.read_html(data)[1]

            all_share = self.get_share(data_df,all_share)

        return all_share

#  ดึงข้อมูลหลักทรัพย์
    def create_dic_info(self,content,stock):

        try:
            data_info,n = {},0
            data_info["Symbol"] = stock
            data = content.split("\n")[1:13]

            while n < 12:

                if n == 4 or n == 8 :
                    n += 2
                    continue        
                data_info[data[n]] = data[n+1]
                n += 2

            return data_info

        except IndexError:

            return {'Symbol': stock,
             'Market': 'Null',
             'Industry': 'Null',
             'Sector': 'Null',
             'Dividend Policy': 'Null'}

    def get_info(self,stock):

        content_1 = self.driver.find_element(By.CSS_SELECTOR,'.company-info-securities').text

        data_info = self.create_dic_info(content_1,stock)

        content_2 = self.driver.find_element(By.CSS_SELECTOR, '.company-info-detail.border.fs-14px.py-4.px-5.mb-4').text
        data = content_2.split("\n")
        data_info[data[0]] = data[1]

        return data_info

    def get_stock_info(self,stock,all_stock):

        stock = stock.split()[0]
        url_target_financial = "https://www.set.or.th/en/market/product/stock/quote/" + stock + "/company-profile/information"
        self.driver.get(url_target_financial)
        self.driver.execute_script("scrollBy(0,+5000);")
        time.sleep(2)
        self.driver.execute_script("scrollBy(0,+5000);")
        time.sleep(3)

        data_info = self.get_info(stock)

        all_stock.append(data_info)

        return all_stock
    
    def crate_Stock_Table(self,stock_name):

        all_stock = []
        i = 0
        for stock in stock_name :

            print(i,' ',stock.split()[0])
            all_stock = self.get_stock_info(stock,all_stock)
            i+=1
        return pd.DataFrame(all_stock)
    
#  ดึงข้อมูล Financial

    def Separate_period_new_period(self,row):

        return row['index'][:8]

    def Separate_period_new_Date(self,row):

        return row['index'][8:]

    def operation_data_Financial(self,stock,dataframe):

        Financial_df = (dataframe.set_index("Period as of")).T
        Financial_df = Financial_df.drop(["Financial Data","Financial Ratio"], axis=1)

        Financial_df['SymbolId'] = self.symbol_id[stock]
        Financial_df = Financial_df.reset_index()
        Financial_df['Period as of'] = Financial_df.apply(lambda row: self.Separate_period_new_period(row), axis=1)
        Financial_df['Date'] = Financial_df.apply(lambda row: self.Separate_period_new_Date(row), axis=1)
        Financial_df['Date'] = pd.to_datetime(Financial_df['Date'])
        del Financial_df['index']

        # Financial_df = Financial_df.rename(columns = {'index':"Period as of"},index = {"Period as of" : " "})
        

        return Financial_df

    def get_Financial_data(self,stock,all_stock_year,all_stock_month):

        stock = stock.split()[0]
        url_target_financial = "https://www.set.or.th/en/market/product/stock/quote/" + stock + "/financial-statement/company-highlights"
        self.driver.get(url_target_financial)

        self.driver.execute_script("scrollBy(0,+5000);")
        time.sleep(3)
        
        Financial_data = self.driver.page_source
        Financial_df = pd.read_html(Financial_data)[0]

        Financial_df = self.operation_data_Financial(stock,Financial_df)

        if 'M/' in (Financial_df.iloc[[-1]]["Period as of"].tolist())[0] :

            all_stock_year.append(Financial_df[:-1])
            all_stock_month.append(Financial_df.iloc[[-1]])

        else : 

            all_stock_year.append(Financial_df)
        
        return all_stock_year,all_stock_month
    
    
    def crate_Financial_Table(self,stock_name):
    
        all_Financial_year = []
        all_Financial_month = []
        
        i = 0
        for stock in stock_name:

            print(i,' ',stock)
            all_Financial_year,all_Financial_month = self.get_Financial_data(stock,all_Financial_year,all_Financial_month)
            i+=1

        table_pice_year = pd.concat(all_Financial_year)
        table_pice_month = pd.concat(all_Financial_month)
        
        return table_pice_year,table_pice_month
    
#  ดึงข้อมูล Statistics

    def operation_data_Statistics(self,stock,dataframe):

        Statistics_df = (dataframe.set_index("Statistics as of")).T

        Statistics_df['SymbolId'] = self.symbol_id[stock]
        Statistics_df = Statistics_df.reset_index()
        Statistics_df = Statistics_df.rename(columns = {"index":"Statistics as of"})

        return Statistics_df

    def get_Statistics_data(self,stock,all_stock_year,all_stock_month):

        stock = stock.split()[0]
        url_target_financial = "https://www.set.or.th/en/market/product/stock/quote/" + stock + "/financial-statement/company-highlights"
        self.driver.get(url_target_financial)
        
        self.driver.execute_script("scrollBy(0,+5000);")
        time.sleep(3)
        
        Statistics_df = self.driver.page_source
        Statistics_df = pd.read_html(Statistics_df)[1]

        Statistics_df = self.operation_data_Statistics(stock,Statistics_df)
       
        if '30' in (Statistics_df.iloc[[-1]]["Statistics as of"].tolist())[0][:2] :

            Statistics_df['Statistics as of'] = pd.to_datetime(Statistics_df['Statistics as of'])
            all_stock_year.append(Statistics_df)

        else : 

            Statistics_df['Statistics as of'] = pd.to_datetime(Statistics_df['Statistics as of'])
            all_stock_year.append(Statistics_df[:-1])
            all_stock_month.append(Statistics_df.iloc[[-1]])
        

        return all_stock_year , all_stock_month
    
    def crate_Statistics_Table(self,stock_name):
    
        all_Statistics_year = []
        all_Statistics_month = []
        i = 0
        for stock in stock_name:

            print(i,' ',stock)
            all_Statistics_year,all_Statistics_month = self.get_Statistics_data(stock,all_Statistics_year,all_Statistics_month)
            i+=1

        table_Statistics_year = pd.concat(all_Statistics_year)
        table_Statistics_month = pd.concat(all_Statistics_month)
        
        return table_Statistics_year,table_Statistics_month

In [12]:
drive = SET_Schap()

C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_22028\3325835944.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(r"C:/Users/JourneyQ/OneDrive - kmutnb.ac.th/Desktop/Quick_file/year_2-S_2/Project_sofedev_2/Schap/chromedriver.exe")


In [13]:
sh_id = drive.get_symbol_id()

In [14]:
sh_id

{'2S': 181,
 '3K-BAT': 112,
 '7UP': 372,
 'A': 224,
 'AAI': 15,
 'AAV': 539,
 'ACC': 373,
 'ACE': 374,
 'ACG': 113,
 'ADVANC': 575,
 'AEONTS': 609,
 'AFC': 69,
 'AGE': 375,
 'AH': 114,
 'AHC': 472,
 'AI': 376,
 'AIE': 377,
 'AIMCG': 283,
 'AIMIRT': 284,
 'AIT': 576,
 'AJ': 161,
 'AJA': 88,
 'AKR': 378,
 'ALLA': 131,
 'ALLY': 285,
 'ALT': 577,
 'ALUCON': 162,
 'AMARIN': 494,
 'AMATA': 225,
 'AMATAR': 286,
 'AMATAV': 226,
 'AMC': 182,
 'AMR': 578,
 'ANAN': 227,
 'AOT': 540,
 'AP': 228,
 'APCO': 100,
 'APCS': 344,
 'APEX': 229,
 'APURE': 16,
 'AQ': 230,
 'AQUA': 495,
 'AS': 496,
 'ASEFA': 132,
 'ASIA': 525,
 'ASIAN': 17,
 'ASIMAR': 541,
 'ASW': 231,
 'AURA': 70,
 'AWC': 232,
 'B': 542,
 'B-WORK': 287,
 'B52': 440,
 'BA': 543,
 'BAFS': 379,
 'BAM': 610,
 'BANPU': 380,
 'BAREIT': 288,
 'BBGI': 381,
 'BBL': 611,
 'BCH': 473,
 'BCP': 382,
 'BCPG': 383,
 'BCT': 146,
 'BDMS': 474,
 'BEAUTY': 441,
 'BEC': 497,
 'BEM': 544,
 'BEYOND': 526,
 'BGC': 163,
 'BGRIM': 384,
 'BH': 475,
 'BIG': 442,
 'BI

In [10]:
stock = (drive.get_SET_share())

# ตารางข้อมูลหุ้น

In [17]:
data_loss = ['BAY', 'BBL', 'CIMBT', 'KBANK', 'KKP', 'KTB', 'LHFG', 'SCB', 'TCAP', 'TISCO', 'TTB', 'AEONTS', 'AMANAH', 'ASAP', 'ASK', 'ASP', 'BAM', 'BYD', 'CGH', 'CHAYO', 'ECL', 'FNS', 'FSS', 'GBX', 'GL', 'HENG', 'IFS', 'JMT', 'KCAR', 'KGI', 'KTC', 'MFC', 'MICRO', 'ML', 'MST', 'MTC', 'NCAP', 'PL', 'S11', 'SAK', 'SAWAD', 'SCAP', 'SGC', 'SM', 'TH', 'THANI', 'TIDLOR', 'TK', 'TNITY', 'UOBKH', 'XPG', 'AYUD', 'BKI', 'BLA', 'BUI', 'CHARAN', 'INSURE', 'KWI', 'MTI', 'NKI', 'NSI', 'SMK', 'TGH', 'THRE', 'THREL', 'TIPH', 'TLI', 'TQM', 'TSI', 'TVI']

In [18]:
df_info = drive.crate_Stock_Table(data_loss)

0   BAY
1   BBL
2   CIMBT
3   KBANK
4   KKP
5   KTB
6   LHFG
7   SCB
8   TCAP
9   TISCO
10   TTB
11   AEONTS
12   AMANAH
13   ASAP
14   ASK
15   ASP
16   BAM
17   BYD
18   CGH
19   CHAYO
20   ECL
21   FNS
22   FSS
23   GBX
24   GL
25   HENG
26   IFS
27   JMT
28   KCAR
29   KGI
30   KTC
31   MFC
32   MICRO
33   ML
34   MST
35   MTC
36   NCAP
37   PL
38   S11
39   SAK
40   SAWAD
41   SCAP
42   SGC
43   SM
44   TH
45   THANI
46   TIDLOR
47   TK
48   TNITY
49   UOBKH
50   XPG
51   AYUD
52   BKI
53   BLA
54   BUI
55   CHARAN
56   INSURE
57   KWI
58   MTI
59   NKI
60   NSI
61   SMK
62   TGH
63   THRE
64   THREL
65   TIPH
66   TLI
67   TQM
68   TSI
69   TVI


In [ ]:
df_info

In [10]:
save_csv(df_info,"Share_info_loss")

'succes'

In [29]:
list(sh_id.keys())[241]

'JWD'

In [31]:
driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe")
i = 0
company = {}
error = []
for share in sh_id.keys():

    try :
        driver.get(f"https://www.set.or.th/en/market/product/stock/quote/{share}/company-profile/information")
        time.sleep(1)
        driver.execute_script("scrollBy(0,+1000);")
        print(i+1)
        full_share = driver.find_element(By.XPATH,'//*[@id="stock-quote-tabs-realty-pane-1"]/div/div[1]/div[1]/div/h2').text
        print(share,' ',full_share)
        company[share] = full_share
        i+=1
    except :
        error.append(share)

C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_22028\3958084249.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe")


1
2S   2S METAL PUBLIC COMPANY LIMITED
2
3K-BAT   THAI ENERGY STORAGE TECHNOLOGY PUBLIC COMPANY LIMITED
3
7UP   SEVEN UTILITIES AND POWER PUBLIC COMPANY LIMITED
4
A   AREEYA PROPERTY PUBLIC COMPANY LIMITED
5
AAI   Asian Alliance International Public Company Limited
6
AAV   ASIA AVIATION PUBLIC COMPANY LIMITED
7
ACC   ADVANCED CONNECTION CORPORATION PUBLIC COMPANY LIMITED
8
ACE   ABSOLUTE CLEAN ENERGY PUBLIC COMPANY LIMITED
9
ACG   AUTOCORP HOLDING PUBLIC COMPANY LIMITED
10
ADVANC   ADVANCED INFO SERVICE PUBLIC COMPANY LIMITED
11
AEONTS   AEON THANA SINSAP (THAILAND) PUBLIC COMPANY LIMITED
12
AFC   ASIA FIBER PUBLIC COMPANY LIMITED
13
AGE   ASIA GREEN ENERGY PUBLIC COMPANY LIMITED
14
AH   AAPICO HITECH PUBLIC COMPANY LIMITED
15
AHC   AIKCHOL HOSPITAL PUBLIC COMPANY LIMITED
16
AI   ASIAN INSULATORS PUBLIC COMPANY LIMITED
17
AIE   AI ENERGY PUBLIC COMPANY LIMITED
18
AIMCG   AIM COMMERCIAL GROWTH FREEHOLD AND LEASEHOLD REAL ESTATE INVESTMENT TRUST
19
AIMIRT   AIM INDUSTRIAL GROWTH FREEHOLD

In [42]:
print(company['LPF'])

LOTUS'S RETAIL GROWTH FREEHOLD AND LEASEHOLD PROPERTY FUND


In [48]:
df = pd.DataFrame(list(company.items()))

In [50]:
df.to_csv('company.csv')

In [46]:
import sqlite3

con = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V2.sqlite')
cur = con.cursor()

for share in company.keys():

        sql = f"""UPDATE Information
                SET Company = "{company[share]}"
                where Symbol = "{share}"
                """
        print(share)
        cur.execute(sql)
        con.commit()

con.close()


2S
3K-BAT
7UP
A
AAI
AAV
ACC
ACE
ACG
ADVANC
AEONTS
AFC
AGE
AH
AHC
AI
AIE
AIMCG
AIMIRT
AIT
AJ
AJA
AKR
ALLA
ALLY
ALT
ALUCON
AMARIN
AMATA
AMATAR
AMATAV
AMC
AMR
ANAN
AOT
AP
APCO
APCS
APEX
APURE
AQ
AQUA
AS
ASEFA
ASIA
ASIAN
ASIMAR
ASW
AURA
AWC
B
B-WORK
B52
BA
BAFS
BAM
BANPU
BAREIT
BBGI
BBL
BCH
BCP
BCPG
BCT
BDMS
BEAUTY
BEC
BEM
BEYOND
BGC
BGRIM
BH
BIG
BIOTEC
BIZ
BJC
BJCHI
BKD
BKKCP
BLA
BLAND
BLISS
BOFFICE
BPP
BR
BRI
BROCK
BRR
BRRGIF
BSBM
BTG
BTNC
BTS
BTSGIF
BWG
CBG
CCET
CCP
CEN
CENTEL
CFRESH
CGD
CH
CHG
CHOTI
CI
CITY
CIVIL
CK
CKP
CM
CMAN
CMC
CMR
CNT
COM7
COTTO
CPALL
CPF
CPH
CPI
CPL
CPN
CPNCG
CPNREIT
CPT
CPTGF
CPW
CRANE
CRC
CSC
CSP
CSR
CSS
CTARAF
CTW
CV
CWT
DCC
DCON
DDD
DELTA
DEMCO
DIF
DMT
DOHOME
DREIT
DRT
DTAC
DTCENT
DTCI
DUSIT
EA
EASON
EASTW
EE
EGATIF
EGCO
EKH
EMC
EP
EPG
ERW
ERWPF
ESSO
ESTAR
ETC
EVER
F&D
FANCY
FE
FMT
FN
FORTH
FPT
FTE
FTI
FTREIT
FUTUREPF
GAHREIT
GC
GEL
GENCO
GFPT
GGC
GIFT
GJS
GLAND
GLOBAL
GLOCON
GPI
GPSC
GRAMMY
GRAND
GREEN
GROREIT
GSTEEL
GULF
GUNKUL
GVREIT
GYT
HANA
HFT
HMPRO
HPF

In [32]:
company

{'2S': '2S METAL PUBLIC COMPANY LIMITED',
 '3K-BAT': 'THAI ENERGY STORAGE TECHNOLOGY PUBLIC COMPANY LIMITED',
 '7UP': 'SEVEN UTILITIES AND POWER PUBLIC COMPANY LIMITED',
 'A': 'AREEYA PROPERTY PUBLIC COMPANY LIMITED',
 'AAI': 'Asian Alliance International Public Company Limited',
 'AAV': 'ASIA AVIATION PUBLIC COMPANY LIMITED',
 'ACC': 'ADVANCED CONNECTION CORPORATION PUBLIC COMPANY LIMITED',
 'ACE': 'ABSOLUTE CLEAN ENERGY PUBLIC COMPANY LIMITED',
 'ACG': 'AUTOCORP HOLDING PUBLIC COMPANY LIMITED',
 'ADVANC': 'ADVANCED INFO SERVICE PUBLIC COMPANY LIMITED',
 'AEONTS': 'AEON THANA SINSAP (THAILAND) PUBLIC COMPANY LIMITED',
 'AFC': 'ASIA FIBER PUBLIC COMPANY LIMITED',
 'AGE': 'ASIA GREEN ENERGY PUBLIC COMPANY LIMITED',
 'AH': 'AAPICO HITECH PUBLIC COMPANY LIMITED',
 'AHC': 'AIKCHOL HOSPITAL PUBLIC COMPANY LIMITED',
 'AI': 'ASIAN INSULATORS PUBLIC COMPANY LIMITED',
 'AIE': 'AI ENERGY PUBLIC COMPANY LIMITED',
 'AIMCG': 'AIM COMMERCIAL GROWTH FREEHOLD AND LEASEHOLD REAL ESTATE INVESTMENT TRUST

# ตาราง Financial

In [19]:
def Change_type_financial(table_financial):
    
    table_financial['Profit (Loss) from Other Activities'] = pd.to_numeric(table_financial['Profit (Loss) from Other Activities'].replace('-',''),errors='coerce')
    table_financial['SymbolId'] = table_financial['SymbolId'].astype('Int64')
    return table_financial

In [20]:
stock = ['ACE', 'ADVANC', 'AEONTS', 'AMATA', 'AOT', 'AP', 'AWC', 'BAM',
       'BANPU', 'BBL', 'BCH', 'BCP', 'BCPG', 'BDMS', 'BEC', 'BEM',
       'BGRIM', 'BH', 'BLA', 'BTS', 'CBG', 'CENTEL', 'CHG', 'CK', 'CKP',
       'COM7', 'CPALL', 'CPF', 'CPN', 'CRC', 'DOHOME', 'DTAC', 'EA',
       'EGCO', 'EPG', 'ESSO', 'FORTH', 'GLOBAL', 'GPSC', 'GULF', 'GUNKUL',
       'HANA', 'HMPRO', 'INTUCH', 'IRPC', 'IVL', 'JMART', 'JMT', 'KBANK',
       'KCE', 'KEX', 'KKP', 'KTB', 'KTC', 'LH', 'MAJOR', 'MEGA', 'MINT',
       'MTC', 'ONEE', 'OR', 'ORI', 'OSP', 'PLANB', 'PSL', 'PTG', 'PTT',
       'PTTEP', 'PTTGC', 'QH', 'RATCH', 'RBF', 'RCL', 'SAWAD', 'SCB',
       'SCC', 'SCGP', 'SINGER', 'SPALI', 'SPRC', 'STA', 'STARK', 'STEC',
       'STGT', 'SUPER', 'SYNEX', 'TASCO', 'TCAP', 'THANI', 'TIDLOR',
       'TIPH', 'TISCO', 'TOP', 'TQM', 'TRUE', 'TTA', 'TTB', 'TU', 'VGI',
       'WHA']

In [21]:
df_Financial_year,df_Financial_month = drive.crate_Financial_Table(stock)

0   ACE
1   ADVANC
2   AEONTS
3   AMATA
4   AOT
5   AP
6   AWC
7   BAM
8   BANPU
9   BBL
10   BCH
11   BCP
12   BCPG
13   BDMS
14   BEC
15   BEM
16   BGRIM
17   BH
18   BLA
19   BTS
20   CBG
21   CENTEL
22   CHG
23   CK
24   CKP
25   COM7
26   CPALL
27   CPF
28   CPN
29   CRC
30   DOHOME
31   DTAC
32   EA
33   EGCO
34   EPG
35   ESSO
36   FORTH
37   GLOBAL
38   GPSC
39   GULF
40   GUNKUL
41   HANA
42   HMPRO
43   INTUCH
44   IRPC
45   IVL
46   JMART
47   JMT
48   KBANK
49   KCE
50   KEX
51   KKP
52   KTB
53   KTC
54   LH
55   MAJOR
56   MEGA
57   MINT
58   MTC
59   ONEE
60   OR
61   ORI
62   OSP
63   PLANB
64   PSL
65   PTG
66   PTT
67   PTTEP
68   PTTGC
69   QH
70   RATCH
71   RBF
72   RCL
73   SAWAD
74   SCB
75   SCC
76   SCGP
77   SINGER
78   SPALI
79   SPRC
80   STA
81   STARK
82   STEC
83   STGT
84   SUPER
85   SYNEX
86   TASCO
87   TCAP
88   THANI
89   TIDLOR
90   TIPH
91   TISCO
92   TOP
93   TQM
94   TRUE
95   TTA
96   TTB
97   TU
98   VGI
99   WHA


In [22]:
df_Financial_month

Period as of,Assets,Liabilities,Equity,Paid-up Capital,Revenue,Profit (Loss) from Other Activities,Net Profit,EPS (Baht),ROA (%),ROE (%),Net Profit Margin (%),SymbolId,Period as of,Date
4,19758.94,5514.6,14186.55,5088.0,5086.59,184.95,1087.95,0.11,9.02,10.73,21.38,374,9M/2022,2022-09-30
4,96164.03,73849.0,21758.49,250.0,15173.06,402.15,3122.19,12.49,7.55,19.94,21.26,609,9M/2023,2022-11-30
4,48278.84,24022.41,19177.37,1150.0,5873.96,102.49,1555.73,1.35,9.6,12.7,39.3,225,9M/2022,2022-09-30
4,63663.7,28090.11,35592.22,3145.9,29842.11,948.99,4722.5,1.5,11.41,17.05,15.82,228,9M/2022,2022-09-30
4,143376.25,61944.04,81432.21,32000.77,6101.35,3617.88,2448.01,0.08,3.84,4.28,40.12,232,9M/2022,2022-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,45660.2,15942.21,26291.75,1822.47,22901.99,136.76,3463.23,1.9,13.72,21.55,14.93,564,9M/2022,2022-09-30
4,1823533.42,1608839.13,214693.16,91852.95,59454.06,1499.97,10347.87,0.11,1.67,6.22,17.4,627,9M/2022,2022-09-30
4,178920.45,115278.45,60875.54,1192.95,116663.96,425.86,5899.84,1.22,5.45,13.29,5.16,65,9M/2022,2022-09-30
4,33350.58,2792.67,27798.61,1119.45,2818.69,-152.36,93.06,0.01,-0.21,-0.13,-2.17,517,6M/2023,2022-09-30


In [23]:
df_Financial_year

Period as of,Assets,Liabilities,Equity,Paid-up Capital,Revenue,Profit (Loss) from Other Activities,Net Profit,EPS (Baht),ROA (%),ROE (%),Net Profit Margin (%),SymbolId,Period as of,Date
0,13886.74,8315.1,5571.64,4579.0,4848.94,-10.04,546.57,0.06,7.52,9.81,11.27,374,Y/E 2018,2018-12-31
1,14145.18,3352.14,10793.04,5088.0,5088.17,-103.51,815.31,0.09,8.74,9.96,16.02,374,Y/E 2019,2019-12-31
2,16429.69,4436.86,11992.83,5088.0,6038.96,6.67,1507.64,0.15,10.84,13.23,24.97,374,Y/E 2020,2020-12-31
3,18740.2,5371.62,13315.33,5088.0,5739.67,167.96,1505.68,0.15,9.95,11.9,26.23,374,Y/E 2021,2021-12-31
0,290504.99,232836.4,57521.04,2973.1,170716.67,-4.34,29682.18,9.98,14.2,55.05,17.41,575,Y/E 2018,2018-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,40848.3,10865.59,29133.39,861.12,4426.15,-194.02,-120.26,-0.01,-0.46,-0.53,-6.24,517,Y/E 2022,2022-03-31
0,78344.95,47330.85,27225.73,1433.43,9079.58,2542.62,2906.81,0.2,7.08,11.08,42.71,281,Y/E 2018,2018-12-31
1,82263.83,49163.25,29477.63,1470.91,11440.12,1945.69,3229.25,0.22,6.54,11.39,34.64,281,Y/E 2019,2019-12-31
2,82621.11,50453.94,28805.19,1494.68,8197.88,1208.76,2523.75,0.17,5.18,8.66,34.95,281,Y/E 2020,2020-12-31


In [24]:
df_Financial_year[df_Financial_year['SymbolId'] == 551]

Period as of,Assets,Liabilities,Equity,Paid-up Capital,Revenue,Profit (Loss) from Other Activities,Net Profit,EPS (Baht),ROA (%),ROE (%),Net Profit Margin (%),SymbolId,Period as of,Date
0,6014.21,3781.04,2233.16,120.00,19894.60,-,1328.55,5.54,27.87,59.49,6.68,551,Y/E 2019,2019-12-31
1,19318.24,8470.76,10847.47,870.00,19010.05,-,1405.03,1.11,14.57,21.48,7.39,551,Y/E 2020,2020-12-31
2,17045.55,7728.66,9298.91,871.29,18972.08,-,46.92,0.03,0.63,0.47,0.24,551,Y/E 2021,2021-12-31


In [25]:
save_csv(df_Financial_year,"save_csv/Financial_en_year")

'succes'

In [26]:
save_csv(df_Financial_month,"save_csv/Financial_en_month")

'succes'

In [27]:
table_financial_year = Change_type_financial(df_Financial_year)
table_financial_month = Change_type_financial(df_Financial_month)

# ตาราง Statistics

In [105]:
def Change_type_satistics(table_satistics):
    
    table_satistics['P/E'] = pd.to_numeric(table_satistics['P/E'].replace('-',''),errors='coerce')
    table_satistics['P/BV'] = pd.to_numeric(table_satistics['P/BV'].replace('-',''),errors='coerce')
    table_satistics['Book Value per share (Baht)'] = pd.to_numeric(table_satistics['Book Value per share (Baht)'].replace('-',''),errors='coerce')
    table_satistics['Dividend Yield (%)'] = pd.to_numeric(table_satistics['Dividend Yield (%)'].replace('-',''),errors='coerce')
    table_satistics['Statistics as of'] = pd.to_datetime(table_satistics['Statistics as of'])
    table_satistics['F/S Period (As of date)'] = pd.to_datetime(table_satistics['F/S Period (As of date)'],errors='coerce')
    table_satistics['SymbolId'] = table_satistics['SymbolId'].astype('Int64')
    
    return table_satistics

In [106]:
df_Statistics_year,df_Statistics_month = drive.crate_Statistics_Table(stock)

0   ACE
1   ADVANC
2   AEONTS
3   AMATA
4   AOT
5   AP
6   AWC
7   BAM
8   BANPU
9   BBL
10   BCH
11   BCP
12   BCPG
13   BDMS
14   BEC
15   BEM
16   BGRIM
17   BH
18   BLA
19   BTS
20   CBG
21   CENTEL
22   CHG
23   CK
24   CKP
25   COM7
26   CPALL
27   CPF
28   CPN
29   CRC
30   DOHOME
31   DTAC
32   EA
33   EGCO
34   EPG
35   ESSO
36   FORTH
37   GLOBAL
38   GPSC
39   GULF
40   GUNKUL
41   HANA
42   HMPRO
43   INTUCH
44   IRPC
45   IVL
46   JMART
47   JMT
48   KBANK
49   KCE
50   KEX
51   KKP
52   KTB
53   KTC
54   LH
55   MAJOR
56   MEGA
57   MINT
58   MTC
59   ONEE
60   OR
61   ORI
62   OSP
63   PLANB
64   PSL
65   PTG
66   PTT
67   PTTEP
68   PTTGC
69   QH
70   RATCH
71   RBF
72   RCL
73   SAWAD
74   SCB
75   SCC
76   SCGP
77   SINGER
78   SPALI
79   SPRC
80   STA
81   STARK
82   STEC
83   STGT
84   SUPER
85   SYNEX
86   TASCO
87   TCAP
88   THANI
89   TIDLOR
90   TIPH
91   TISCO
92   TOP
93   TQM
94   TRUE
95   TTA
96   TTB
97   TU
98   VGI
99   WHA


In [107]:
df_Statistics_year

Statistics as of,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),SymbolId
0,2019-12-30,4.22,42942.72,-,62.46,-,-,-,374
1,2020-12-30,3.64,37040.64,30 Sep 2020,24.71,3.15,1.15,-,374
2,2021-12-30,3.50,35616.00,30 Sep 2021,25.58,2.75,1.27,1.37,374
3,2022-12-30,2.70,27475.20,30 Sep 2022,18.75,1.92,1.40,0.75,374
0,2019-12-30,213.00,633287.05,30 Sep 2019,20.45,10.22,20.84,3.32,575
...,...,...,...,...,...,...,...,...,...
3,2022-12-30,4.40,49255.88,30 Sep 2022,-,1.77,2.48,0.80,517
0,2019-12-30,3.88,57071.30,30 Sep 2019,16.36,1.90,2.04,2.10,281
1,2020-12-30,3.04,45438.38,30 Sep 2020,20.01,1.61,1.89,4.40,281
2,2021-12-30,3.52,52612.86,30 Sep 2021,26.19,1.82,1.93,2.85,281


In [39]:
df_Statistics_year["SymbolId"] = df_Statistics_year["Symbol"].map(sh_id)

In [108]:
df_Statistics_month

Statistics as of,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),SymbolId
4,2023-01-23,2.64,26864.64,30 Sep 2022,18.27,1.87,1.41,0.77,374
4,2023-01-23,201.00,597816.16,30 Sep 2022,23.43,7.66,26.25,3.83,575
4,2023-01-23,189.50,47375.00,30 Nov 2022,11.75,2.18,87.03,2.72,609
4,2023-01-23,20.40,23460.00,30 Sep 2022,10.22,1.22,16.68,1.96,225
4,2023-01-23,74.75,1067856.08,30 Sep 2022,-,10.54,7.09,-,540
...,...,...,...,...,...,...,...,...,...
4,2023-01-23,8.10,14761.96,30 Sep 2022,2.96,0.56,14.43,2.72,564
4,2023-01-23,1.44,139357.78,30 Sep 2022,10.60,0.65,2.22,2.63,627
4,2023-01-23,16.30,77780.59,30 Sep 2022,9.69,1.25,13.08,5.83,65
4,2023-01-23,4.82,53957.58,30 Sep 2022,-,1.94,2.48,0.73,517


In [40]:
df_Statistics_month["SymbolId"] = df_Statistics_month["Symbol"].map(sh_id)

In [41]:
del df_Statistics_year["Symbol"]
del df_Statistics_month["Symbol"]

In [109]:
save_csv(df_Statistics_year,"Statistics_en_year")

'succes'

In [110]:
save_csv(df_Statistics_month,"Statistics_en_month")

'succes'

In [112]:
table_satistics_year = Change_type_satistics(df_Statistics_year)
table_satistics_day = Change_type_satistics(df_Statistics_month)